In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("data/BCS.csv", index_col = 0)
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-06-05,154.199997,154.199997,151.059998,152.500000,130.843079,67049984.0
2019-06-06,153.339996,153.520004,149.600006,151.240005,129.762009,31278842.0
2019-06-07,151.679993,152.559998,150.199997,151.059998,129.607559,21154119.0
2019-06-10,151.979996,153.740005,150.699997,153.080002,131.340698,21160315.0
2019-06-11,152.619995,154.059998,151.520004,152.220001,130.602844,22900404.0
...,...,...,...,...,...,...
2024-05-30,212.550003,219.850006,211.250000,218.550003,218.550003,47795152.0
2024-05-31,219.199997,221.100006,217.199997,220.000000,220.000000,82218581.0
2024-06-03,222.050003,224.300003,219.000000,219.199997,219.199997,47887792.0


In [5]:

1 - (df['Adj Close'].pct_change() + 1).cumprod().tail()[0]

/var/folders/pf/r0cmnchs509dgvhzjm4qsn640000gn/T/ipykernel_33109/628832764.py:1: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  1 - (df['Adj Close'].pct_change() + 1).cumprod().tail()[0]
/var/folders/pf/r0cmnchs509dgvhzjm4qsn640000gn/T/ipykernel_33109/628832764.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  1 - (df['Adj Close'].pct_change() + 1).cumprod().tail()[0]


-0.6703214619399178

In [6]:
from typing import Type
import boto3
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
import streamlit as st
import pandas as pd

## We will be suing Titan Embeddings Model To generate Embedding
from langchain.embeddings.bedrock import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
from langchain.chat_models.bedrock import BedrockChat

## Data Ingestion
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Vector Embedding And Vector Store
from langchain.vectorstores import FAISS

## LLm Models
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [7]:

from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain_community.callbacks.streamlit import (
    StreamlitCallbackHandler,
)
from langchain_core.documents import Document

In [13]:
bedrock = boto3.client(service_name="bedrock-runtime")
bedrock_embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-image-v1", client=bedrock
)

In [14]:
def get_llama2_llm():
    ##create the Llama Model
    llm=Bedrock(model_id="meta.llama2-70b-chat-v1",client=bedrock,
                model_kwargs={'max_gen_len':200})
    
    return llm

In [15]:
vectorstore = FAISS.load_local(
        "faiss_index_reports", bedrock_embeddings, allow_dangerous_deserialization=True
    )

In [47]:
def search_docs(query):
    print(query)
    """Searches the document store for relevant information."""
    vectorstore = configure_retriever()
    # print(f"query: {query}")
    # if query["value"]:
    #     results = vectorstore.similarity_search(query["value"])
    # else:
    #     results = vectorstore.similarity_search(query["query"])
    results = vectorstore.similarity_search(query)
    return {"docs": results}


retriever_tool = Tool(
    name="search_docs",
    func=search_docs,
    description="Search the document store for relevant information.",
)

In [48]:
tools = [retriever_tool]
llm = get_llama2_llm()
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=ConversationBufferMemory(),
    handle_parsing_error = True,
    verbose=True,
)

In [49]:
st_callback = StreamlitCallbackHandler(st.container())

In [50]:
def configure_retriever():
    bedrock_embeddings = BedrockEmbeddings(
        model_id="amazon.titan-embed-image-v1", client=bedrock
    )
    vectorstore = FAISS.load_local(
        "faiss_index_pdfs", bedrock_embeddings, allow_dangerous_deserialization=True
    )
    return vectorstore

In [51]:
response = agent.invoke(
                {"input": "Hello"}, {"callbacks": [st_callback]}
            )
            



> Entering new AgentExecutor chain...


Error in StreamlitCallbackHandler.on_llm_end callback: StreamlitAPIException()
Error in StreamlitCallbackHandler.on_tool_start callback: StreamlitAPIException()


Action:
```
{
  "action": "search_docs",
  "action_input": "human greetings"
}
```
human greetings


Error in StreamlitCallbackHandler.on_tool_end callback: StreamlitAPIException()



Observation: {'docs': [Document(page_content='3  \nExecutive Summary', metadata={'source': 'pdfs/The-CECL-Accounting-Standard-and-Financial-Institution-Regulatory-Capital-Study-9-15-20.pdf', 'page': 2}), Document(page_content='6  \nI.  Background', metadata={'source': 'pdfs/The-CECL-Accounting-Standard-and-Financial-Institution-Regulatory-Capital-Study-9-15-20.pdf', 'page': 5}), Document(page_content='25  \nIV.  Recommendations', metadata={'source': 'pdfs/The-CECL-Accounting-Standard-and-Financial-Institution-Regulatory-Capital-Study-9-15-20.pdf', 'page': 24}), Document(page_content='is otherwise not r estricted b y the ca pital rule’s automa tic distrib ution r estrictions , beginning\non October 1, 2020, to\n(i) ma ke capital distrib utions in e xcess of those specif ied in the ca pital plan in a gi ven\nquarter and\n(ii) execute distrib utions origina lly planned f or a la ter quarter in the planning horizon in an\nearlier quarter , in either case up to the limit of the automa tic 

...

 It seems like the human is asking about the CECL accounting standard and its impact on financial institutions. They also mention the Federal Reserve's instructions for the 2020 Comprehensive Capital Analysis and Review (CCAR) and the stress capital buffer.

Action:
```
{
  "action": "search_docs",
  "action_input": "CECL accounting standard, Federal Reserve instructions, CCAR 2020, stress capital buffer"
}
```

CECL accounting standard, Federal Reserve instructions, CCAR 2020, stress capital buffer


Error in StreamlitCallbackHandler.on_tool_end callback: StreamlitAPIException()



Observation: {'docs': [Document(page_content='17 B. Key Regulatory Responses Related to Regulatory Capital  \nThe prudential regulators have acknowledged many of the potential challenges and effects of \nfinancial institutions’ implementation of CECL and have taken various actions in response , \ndescribed below .  \n1. 2018 Fed eral Reserve  Stress Test Statement  \nOn December 21, 2018, the Federal Reserve announced that it planned to maintain the current \nmodeling framework for loan allowances in its supervisory stress test process through 2021 to \nprovide more time for it to analyze the potential impacts of CECL.49  The Federal Reserve chose \nto maintain the current framework in supervisory stress tests in order to reduce uncertainty and \nallow for better capital planning at affected banking organi zations, among other reasons.50  In \naddition, as part of the Banking Regulators’ 2019 Regulatory Capital Rule, the Federal Reserve \namended its stress testing rules to require a 

Error in StreamlitCallbackHandler.on_llm_error callback: StreamlitAPIException()


ValueError: Error raised by bedrock service: An error occurred (ValidationException) when calling the InvokeModel operation: This model's maximum context length is 4096 tokens. Please reduce the length of the prompt

In [46]:
vectorstore.similarity_search("Hello")

[Document(page_content='The Barclays Bank UK Board approves the Barclays Bank UK Group funding plan, internal stress tests, regulatory stress test results, and the \nBarclays Bank UK contribution to the Barclays Group recovery plan. The Barclays Bank UK PLC ALCO is responsible for monitoring and \nmanaging liquidity risk in line with the Barclays Bank UK Group’s funding management objectives, funding plan and risk framework. The \nBarclays UK Risk Committee monitors and reviews the liquidity risk profile and control environment, providing second line oversight of the \nmanagement of liquidity risk. The Barclays Bank UK PLC Board Risk Committee reviews the risk profile, and reviews liquidity risk appetite at \nleast annually and the impact of stress scenarios on the Barclays Bank UK Group funding plan/forecast in order to agree the Barclays Bank \nUK Group’s projected funding abilities.\nCapital risk management (audited)\nOverview\nCapital risk is managed through ongoing monitoring and 

In [26]:
faiss_index = FAISS.load_local("faiss_index_pdfs", bedrock_embeddings, allow_dangerous_deserialization=True)
llm=get_llama2_llm()

In [32]:
prompt_template = """

Human: Use the following pieces of context to provide a 
concise answer to the question at the end but usse atleast summarize with 
100 words with detailed explantions. If you don't know the answer, 
just say that you don't know, don't try to make up an answer.
<context>
{context}
</context

Question: {question}

Assistant:"""

In [33]:
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
PROMPT

PromptTemplate(input_variables=['context', 'question'], template="\n\nHuman: Use the following pieces of context to provide a \nconcise answer to the question at the end but usse atleast summarize with \n100 words with detailed explantions. If you don't know the answer, \njust say that you don't know, don't try to make up an answer.\n<context>\n{context}\n</context\n\nQuestion: {question}\n\nAssistant:")

In [36]:

qa = RetrievalQA.from_chain_type(
llm=llm,
chain_type="stuff",
retriever=faiss_index.as_retriever(
    search_type="similarity", search_kwargs={"k": 3}
),
return_source_documents=False,
chain_type_kwargs={"prompt": PROMPT}
)
answer=qa({"query":"Hello"})

In [37]:
answer

{'query': 'Hello',
 'result': " Hi there! I'm here to help you with any questions you have. What's your question?"}

In [52]:
from pypdf import PdfReader



In [54]:
reader = PdfReader("pdfs/CCAR-QAs.pdf")
page = reader.pages[0]
print(page.extract_text())

The F ederal Reserv e conducts the ann ual Compr ehensi ve Capital and Ana lysis R eview (CCAR)
exercise to assess ca pital positions and planning pr actices of large firms consistent with R egula-
tion YY (12 CFR part 252) and the ca pital plan rule (12 CFR 225.8).1The F ederal Reserv e
conducts a quantita tive assessment of firms’ ca pital positions in CCAR using the Dod d-Frank
Act str ess tests (DF AST) as implemented in R egula tion YY .2The F ederal Reserv e also collects
information r elevant to the CCAR and DF AST e xercises on the FR Y -14 r eports .3
Federal Reserv e staf f provides ans wers to questions fr om f irms related to CCAR and DF AST
on an ongoing basis to assist with the interpr etation of reporting instructions and r elated r egu-
lations and supervisory guidance . These questions and ans wers ar e provided dir ectly to a ll firms
subject to CCAR and DF AST and pub lished on the F ederal Reserv e’s w ebsite to facilita te
transpar ency and consistency of interpr e

In [59]:
loader=PyPDFDirectoryLoader("guidelines")
documents=loader.load()

# - in our testing Character split works better with this PDF data set
text_splitter=RecursiveCharacterTextSplitter(chunk_size=10000,
                                                chunk_overlap=1000)

docs=text_splitter.split_documents(documents)

In [68]:
docs[10].page_content

'Global Market Shock\nFirms with significant trading activity must include\nin their company-run stress tests under the supervi-\nsory severely adverse scenario a component that\nassesses potential losses associated with trading posi-\ntions, private equity positions, and counterparty\nexposures (global market shock). The Board modi-\nfied the threshold for the global market shock com-\nponent in 2017 to apply to any firm that is subject to\nsupervisory stress tests and that has aggregate trad-\ning assets and liabilities of $50 billion or more, or\naggregate trading assets and liabilities equal to\n10 percent or more of total consolidated assets, and\nis not a large and noncomplex firm.25\nFirms subject to the global market shock in CCAR\n2020 must apply the shock as of a specified point in\ntime, which will result in instantaneous losses and a\nreduction in capital. These losses and related capital\nimpact will be included in projections for the first\nquarter of the planning horizon

In [62]:
len(docs)

26

In [69]:
from langchain.text_splitter import MarkdownTextSplitter

In [70]:
splitter = MarkdownTextSplitter(chunk_size = 40, chunk_overlap=0)


## Header1
### Header 2